async/awaitの使い方をメモる

[参考](https://fisproject.jp/2018/12/coroutines-with-async-and-await/)

yieldによって作られたコルーチンをジェネレータ型のコルーチン。
async/awaitによって作られたコルーチンをネイティブ型のコルーチンと呼ぶ、らしい。

In [1]:
class averager:
    def __await__(self):
        total = 0
        count = 0
        average = None
        while True:
            term = yield
            if term is None:
                break
            total += term
            count += 1
            average = total / count
        return count, average

In [3]:
async def grouper(results, key):
    while True:
        results[key] = await averager()

In [13]:
data = {
    "a": range(10),
    "b": range(20)
}

results = {}
for k in data:
    g = grouper(results, k)
    g.send(None) # send()ではダメ
    for v in data[k]:
        g.send(v)
    g.send(None)
print(results)

{'a': (10, 4.5), 'b': (20, 9.5)}


ポイント：
* `async def`でネイティブコルーチン関数が作られる
* ネイティブコルーチン関数は、`await`の部分で、実行を一時停止できる
* `await`の右側には `awaitable` な値が来る
  * `awaitable`な値とは`__await__`をもつオブジェクト
  
実行を一時停止できるコードの書き方は分かった。一時停止して処理を別の部分に移す方法が知りたい。  